<a href="https://colab.research.google.com/github/sprerak48/Autonomous-tagging-using-Deep-Learning/blob/Prerak/LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Ques.csv to Ques.csv


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian") 

In [0]:
df = pd.read_csv('Ques.csv', sep = ',', names = ['Title', 'Body'], error_bad_lines=False)

In [0]:
df.describe()



In [0]:
df.head(20)

In [0]:
def clean_Body(Body):
    
    ## Remove puncuation
    Body = Body.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    Body = Body.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    Body = [w for w in Body if not w in stops and len(w) >= 3]
    
    Body = " ".join(Body)

    # Clean the Body
    Body = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", Body)
    Body = re.sub(r"what's", "what is ", Body)
    Body = re.sub(r"\'s", " ", Body)
    Body = re.sub(r"\'ve", " have ", Body)
    Body = re.sub(r"n't", " not ", Body)
    Body = re.sub(r"i'm", "i am ", Body)
    Body = re.sub(r"\'re", " are ", Body)
    Body = re.sub(r"\'d", " would ", Body)
    Body = re.sub(r"\'ll", " will ", Body)
    Body = re.sub(r",", " ", Body)
    Body = re.sub(r"\.", " ", Body)
    Body = re.sub(r"!", " ! ", Body)
    Body = re.sub(r"\/", " ", Body)
    Body = re.sub(r"\^", " ^ ", Body)
    Body = re.sub(r"\+", " + ", Body)
    Body = re.sub(r"\-", " - ", Body)
    Body = re.sub(r"\=", " = ", Body)
    Body = re.sub(r"'", " ", Body)
    Body = re.sub(r"(\d+)(k)", r"\g<1>000", Body)
    Body = re.sub(r":", " : ", Body)
    Body = re.sub(r" e g ", " eg ", Body)
    Body = re.sub(r" b g ", " bg ", Body)
    Body = re.sub(r" u s ", " american ", Body)
    Body = re.sub(r"\0s", "0", Body)
    Body = re.sub(r" 9 11 ", "911", Body)
    Body = re.sub(r"e - mail", "email", Body)
    Body = re.sub(r"j k", "jk", Body)
    Body = re.sub(r"\s{2,}", " ", Body)
    
    Body = Body.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in Body]
    Body = " ".join(stemmed_words)

    return Body

In [0]:
df['Body'] = df['Body'].map(lambda x: clean_Body(x))


AttributeError: ignored

In [0]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(250000, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv
  


In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../content"))

# Any results you write to the current directory are saved as output.

import nltk
nltk.download('wordnet')
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, CuDNNLSTM, CuDNNGRU, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, SpatialDropout1D, concatenate
from keras.models import Model, Sequential, load_model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D, GlobalAveragePooling1D
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from nltk.stem import WordNetLemmatizer
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

['.config', 'Ques.csv', 'sample_data']
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
train_f = pd.read_csv("Ques.csv")
test_f = pd.read_csv("Ques.csv")
corpus_sentences = list(map(str,train_f["Title"])) + list(map(str,test_f["Title"]))

In [0]:
def clean_text(c):
    lemmatizer = WordNetLemmatizer()
    lemmed_words = c.copy()
    i = 0
    for sentences in c:
        temp = [lemmatizer.lemmatize(j) for j in sentences.lower().split()]
        lemmed_words[i] = " ".join(temp)
        i+=1
    text = lemmed_words.copy()
    return(text)

text = clean_text(corpus_sentences)

In [0]:
X_train = train_f['Title']
y_train = train_f['Body']
Xy_train = pd.concat([X_train, y_train], axis=1)

In [0]:
Xy_train['Title'] = clean_text(list(map(str,Xy_train["Title"])))
test_f['Title'] = clean_text(list(map(str,test_f["Title"])))

In [0]:
max_words = 15000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(text))

In [0]:
list_tokenized_train = tokenizer.texts_to_sequences(Xy_train["Title"])
list_tokenized_test = tokenizer.texts_to_sequences(test_f["Title"])

In [0]:
max_len = 80
X_train_final = pad_sequences(list_tokenized_train,maxlen=max_len)
X_test_final = pad_sequences(list_tokenized_test,maxlen=max_len)

In [0]:
train_dummies = pd.get_dummies(Xy_train['Body'])
y_train_final = train_dummies.values

In [0]:
np.random.seed(226)
shuffle_indices = np.random.permutation(np.arange(len(X_train_final)))
X_trains = X_train_final[shuffle_indices]
y_trains = y_train_final[shuffle_indices]

In [0]:
phs = Xy_train['Title'][shuffle_indices]

In [0]:
td = 5
vec = TfidfVectorizer(max_features=td, ngram_range=(1,2))
x_tfidf = vec.fit_transform(phs).toarray()
test_tfidf = vec.transform(test_f['Title']).toarray()

In [0]:
def keras_dl(model, embed_size, batch_size, epochs):   
    inp = Input(shape = (max_len,), name = 'lstm')
    x = Embedding(max_words,embed_size,weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(0.5)(x)
    
    x_lstm = Bidirectional(CuDNNLSTM(128, return_sequences = True))(x1)
    x_lstm_c1d = Conv1D(64,kernel_size=3,padding='valid',activation='tanh')(x_lstm)
    x_lstm_c1d_gp = GlobalMaxPooling1D()(x_lstm_c1d)
    
    x_gru = Bidirectional(CuDNNGRU(128, return_sequences = True))(x1)
    x_gru_c1d = Conv1D(64,kernel_size=2,padding='valid',activation='tanh')(x_gru)
    x_gru_c1d_gp = GlobalMaxPooling1D()(x_gru_c1d)

    inp2 = Input(shape = (td,), name = 'tfidf')
    x2 = BatchNormalization()(inp2)
    x2 = Dense(8, activation='tanh')(x2)
    
    x_f = concatenate([x_lstm_c1d_gp, x_gru_c1d_gp])
    x_f = BatchNormalization()(x_f)
    x_f = Dropout(0.4)(Dense(128, activation='tanh') (x_f))    
    x_f = BatchNormalization()(x_f)
    x_f = concatenate([x_f, x2])
    x_f = Dropout(0.4)(Dense(64, activation='tanh') (x_f))
    x_f = Dense(5, activation = "sigmoid")(x_f)
    model = Model(inputs = [inp, inp2], outputs = x_f)
       
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return (model)

In [36]:
embed_size = 300
batch_size = 8
epochs = 3
model = Sequential()

file_path = "test_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)

firstmodel = keras_dl(model, embed_size, batch_size, epochs)

text_model = firstmodel.fit({'lstm': X_trains, 'tfidf': x_tfidf}, y_trains, batch_size=batch_size,epochs=epochs,verbose=0,
                            validation_split = 0.1,
                            callbacks = [check_point, early_stop])

firstmodel = load_model(file_path)
pred = firstmodel.predict([np.array(X_test_final), test_tfidf], verbose = 1)
pred2 = np.round(np.argmax(pred, axis=1)).astype(int)

sub = pd.DataFrame({'Title': test_f['Title'],
                   'Body': pred2})

sub.to_csv("SF.csv", index = False, header = True)

ValueError: ignored